In [10]:
import re
from pyvi import ViTokenizer
import pandas as pd

# Hàm tiền xử lý

In [11]:
ACRONYMS = {
    "r": "rồi", "a": "anh", "e": "em", "j": "gì", "k": "không", "m": "mình",
    "t": "tôi", "b": "bạn", "h": "giờ", "s": "sao",
    "ko": "không", "k0": "không", "bt": "bình thường", "vn": "việt nam", "vs": "và",
    "cx": "cũng được", "đc": "được", "dc": "được", "nh": "nhưng", "đg": "đường",
    "nc": "nước", "ms": "mới", "bh": "bao giờ", "km": "khuyến mãi", "ae": "anh em",
    "sg": "sài gòn", "hn": "hà nội", "vk": "vợ", "ck": "chồng", "nv": "nhân viên",
    "mn": "mọi người", "qc": "quảng cáo", "sp": "sản phẩm", "sd": "sử dụng", "wa": "quá",
    "ntn": "như thế nào", "lun": "luôn", "trc": "trước", "chx": "chưa", "nhg": "nhưng",
    "spt": "sản phẩm tốt", "nch": "nói chung", "kbh": "không bao giờ",
    "tnao": "thế nào", "hqua": "hôm qua", "toẹt": "tuyệt", "nhưg": "nhưng",
    "hnay": "hôm nay", "thik": "thích",
    "lquan": "liên quan", "nchung": "nói chung", "k_thể": "không thể"
}

# compile patterns for performance: sort keys by length desc to avoid partial matches
_acronyms_sorted = sorted(ACRONYMS.keys(), key=len, reverse=True)
_acro_pattern = re.compile(r'\b(' + '|'.join(re.escape(k) for k in _acronyms_sorted) + r')\b')

def replace_acronyms(text):
    return _acro_pattern.sub(lambda m: ACRONYMS[m.group(1)], text)

# simple emoji remover (covers common ranges)
_emoji_re = re.compile(
    "[" 
    "\U0001F600-\U0001F64F"  # emoticons
    "\U0001F300-\U0001F5FF"  # symbols & pictographs
    "\U0001F680-\U0001F6FF"  # transport & map
    "\U0001F1E0-\U0001F1FF"  # flags
    "\U00002700-\U000027BF"
    "\U0001F900-\U0001F9FF"
    "\U0001FA70-\U0001FAFF"
    "]+", flags=re.UNICODE)

def remove_emoji(text):
    return _emoji_re.sub('', text)

# small normalization map (kept minimal)
_norm_map = {
    'òa': 'oà', 'óa': 'oá', 'ỏa': 'oả', 'õa': 'oã', 'ọa': 'oạ',
    'ủ': 'ủ', 'a´': 'á', 'aˋ': 'à'
}

_norm_re = re.compile('|'.join(re.escape(k) for k in _norm_map.keys()))
def normalize_vietnamese(text):
    text = _norm_re.sub(lambda m: _norm_map[m.group(0)], text)
    # reduce repeated letters: đẹpppp -> đẹp
    return re.sub(r'([a-zA-Zà-ỹ])\1{2,}', r'\1', text)

def remove_special_characters(text):
    # remove urls, mentions, hashtags then keep letters + spaces
    text = re.sub(r"http\S+|www\S+|@\S+|#\S+", " ", text)
    return re.sub(r"[^a-zA-ZÀ-ỹà-ỹ\s]", " ", text)

def preprocess_text(text):
    text = text.lower().replace("\n", " ").replace("\t", " ").strip()
    text = remove_emoji(text)
    text = normalize_vietnamese(text)
    text = remove_special_characters(text)
    text = replace_acronyms(text)
    return re.sub(r'\s+', ' ', text).strip()


In [12]:
df = pd.read_csv('coopmart_review.csv')
df

,title,publishedAtDate,originalLanguage,text,textTranslated,stars,comment
0,Co.opmart Bình Triệu,2025-04-24T17:45:42.757Z,vi,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch...",NaN,5,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch..."
1,Co.opmart Bình Triệu,2025-04-07T04:12:59.369Z,vi,Dịch vụ tốt,NaN,5,Dịch vụ tốt
2,Co.opmart Bình Triệu,2025-03-30T13:47:13.978Z,vi,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t...",NaN,1,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t..."
3,Co.opmart Bình Triệu,2025-03-24T14:19:32.053Z,vi,Nhân viên tính tiền thái độ khó chịu ra mặt,NaN,1,Nhân viên tính tiền thái độ khó chịu ra mặt
4,Co.opmart Bình Triệu,2025-03-02T14:42:40.200Z,vi,Thật tuyệt,NaN,5,Thật tuyệt
...,...,...,...,...,...,...,...
31974,Co.opmart Cao Lãnh,2021-08-15T08:44:18.773Z,vi,Tốt,NaN,4,Tốt
31975,Co.opmart Cao Lãnh,2019-04-08T08:23:04.140Z,vi,Tốt,NaN,4,Tốt
31976,Co.opmart Cao Lãnh,2019-04-13T11:10:38.372Z,vi,Tốt,NaN,4,Tốt
31977,Co.opmart Cao Lãnh,2019-09-28T12:38:48.364Z,vi,Tuyệt,NaN,4,Tuyệt


In [13]:
df['comment'] = df['comment'].apply(preprocess_text)
df = df.dropna(subset=['comment'])
df = df[df['comment'].str.strip() != ''] 
df

,title,publishedAtDate,originalLanguage,text,textTranslated,stars,comment
0,Co.opmart Bình Triệu,2025-04-24T17:45:42.757Z,vi,"Nhiều hàng hoá để lựa chọn, thường xuyên có ch...",NaN,5,nhiều hàng hoá để lựa chọn thường xuyên có chư...
1,Co.opmart Bình Triệu,2025-04-07T04:12:59.369Z,vi,Dịch vụ tốt,NaN,5,dịch vụ tốt
2,Co.opmart Bình Triệu,2025-03-30T13:47:13.978Z,vi,"Hôm nay ngày 30-3 lúc tầm 19h56 -20h30 , tôi t...",NaN,1,hôm nay ngày lúc tầm giờ giờ tôi tính tiền ở q...
3,Co.opmart Bình Triệu,2025-03-24T14:19:32.053Z,vi,Nhân viên tính tiền thái độ khó chịu ra mặt,NaN,1,nhân viên tính tiền thái độ khó chịu ra mặt
4,Co.opmart Bình Triệu,2025-03-02T14:42:40.200Z,vi,Thật tuyệt,NaN,5,thật tuyệt
...,...,...,...,...,...,...,...
31974,Co.opmart Cao Lãnh,2021-08-15T08:44:18.773Z,vi,Tốt,NaN,4,tốt
31975,Co.opmart Cao Lãnh,2019-04-08T08:23:04.140Z,vi,Tốt,NaN,4,tốt
31976,Co.opmart Cao Lãnh,2019-04-13T11:10:38.372Z,vi,Tốt,NaN,4,tốt
31977,Co.opmart Cao Lãnh,2019-09-28T12:38:48.364Z,vi,Tuyệt,NaN,4,tuyệt


In [14]:
df = df[['comment',"stars"]]

In [15]:
df.to_csv('coopmart_review_preprocess.csv', encoding='utf-8-sig', index=False)